To Fix, must change Book number to last one in BettingPros (i.e. Book7, Book12) for Anytime TD accordingly. Must change matchup slugs week to week in order to acquire the new CSV projection. Must change SEASON, WEEK variables. 

In [1]:
# === BettingPros Consensus scraper (text-block method) ===
# Logic: split on "View Matchup" → parse ALL O/U pairs in each block → take LAST valid pair (Consensus).

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time, re
import pandas as pd
from pathlib import Path

# ====== CONFIG ======
SEASON = 2025
WEEK = 1

URLS = [
    ("REC",      f"https://www.bettingpros.com/nfl/odds/player-props/receptions/?season={SEASON}&week={WEEK}"),
    ("RUSH YDS", f"https://www.bettingpros.com/nfl/odds/player-props/rushing-yards/?season={SEASON}&week={WEEK}"),
    ("REC YDS",  f"https://www.bettingpros.com/nfl/odds/player-props/receiving-yards/?season={SEASON}&week={WEEK}"),
]

# Your chromedriver path
driver_path = "/Users/nicholashazzard/Downloads/chromedriver"

out_dir = Path(f"bettingpros_week{WEEK}_{SEASON}")
out_dir.mkdir(parents=True, exist_ok=True)

# ====== SELENIUM ======
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

def scroll_to_bottom_safely(driver, pause=2, max_loops=40):
    last_height = driver.execute_script("return document.body.scrollHeight")
    loops = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height or loops >= max_loops:
            break
        last_height = new_height
        loops += 1

# ====== Parsing helpers (layout-aware) ======
POS_ABBR = r"(QB|RB|WR|TE|DST)"
TEAM_POS_RE = re.compile(rf"^[A-Z]{{2,3}}\s*-\s*{POS_ABBR}\b")

def _norm(s: str) -> str:
    return (s or "").replace("−", "-")  # normalize unicode minus

def _odds_to_int(tok: str):
    tok = tok.strip().upper()
    if tok == "EVEN":
        return 100
    return int(_norm(tok))

# Grab (line, over_odds, under_odds) sequences anywhere in text (handles same-line or next-line parentheses)
# We'll scan line-by-line to be tolerant of formatting breaks.
ODDS_PAT_INLINE = re.compile(r"^\s*[OU]\s*([0-9]+(?:\.[0-9]+)?)\s*\((EVEN|[-+]?\d+)\)\s*$", re.I)
ODDS_PAT_TWO = re.compile(r"^\s*[OU]\s*([0-9]+(?:\.[0-9]+)?)\s*$", re.I)
PAREN_PAT = re.compile(r"\((EVEN|[-+]?\d+)\)")

def extract_all_pairs(block_text: str):
    """
    Returns a list of (line_value, over_odds, under_odds) in the order they appear.
    Skips NL/OFF segments.
    """
    lines = [ln for ln in block_text.splitlines() if ln.strip() != ""]
    pairs = []
    i = 0
    while i < len(lines):
        s = lines[i].strip()
        s_up = s.upper()
        # skip NL / OFF
        if s_up in ("NL", "OFF"):
            i += 1
            continue

        # Detect Over
        m_inline = ODDS_PAT_INLINE.match(_norm(s))
        over_line = over_odds = None
        if m_inline and s.strip().upper().startswith("O"):
            over_line = float(m_inline.group(1))
            over_odds = _odds_to_int(m_inline.group(2))
            i += 1
        else:
            m_two = ODDS_PAT_TWO.match(s)
            if m_two and s.strip().upper().startswith("O"):
                over_line = float(m_two.group(1))
                # look ahead for odds parens within next 2 lines
                over_odds = None
                look = 1
                while over_odds is None and look <= 2 and (i+look) < len(lines):
                    mpar = PAREN_PAT.search(_norm(lines[i+look]))
                    if mpar:
                        over_odds = _odds_to_int(mpar.group(1))
                    look += 1
                i += 1
            else:
                i += 1
                continue  # not an Over line

        # Find the next Under
        under_line = under_odds = None
        j = i
        while j < len(lines):
            t = lines[j].strip()
            t_up = t.upper()
            if t_up in ("NL", "OFF"):
                j += 1
                continue
            m_inline_u = ODDS_PAT_INLINE.match(_norm(t))
            if m_inline_u and t.strip().upper().startswith("U"):
                under_line = float(m_inline_u.group(1))
                under_odds = _odds_to_int(m_inline_u.group(2))
                j += 1
                break
            m_two_u = ODDS_PAT_TWO.match(t)
            if m_two_u and t.strip().upper().startswith("U"):
                under_line = float(m_two_u.group(1))
                # odds may be on same or next lines
                look = 1
                while under_odds is None and look <= 2 and (j+look) < len(lines):
                    mpar = PAREN_PAT.search(_norm(lines[j+look]))
                    if mpar:
                        under_odds = _odds_to_int(mpar.group(1))
                    look += 1
                j += 1
                break
            j += 1

        # Validate and record
        if over_line is not None and under_line is not None and over_odds is not None and under_odds is not None:
            # Use only matching-line O/U markets (same posted number)
            if abs(over_line - under_line) < 1e-9:
                pairs.append((over_line, over_odds, under_odds))

        i = j  # continue after the U-segment we consumed (or continue scan)

    return pairs

def extract_player_teampos(block_text: str):
    """
    From the block, grab the first TEAM - POS line and the player just above it.
    """
    lines = [ln for ln in block_text.splitlines() if ln.strip() != ""]
    for idx in range(len(lines)):
        if TEAM_POS_RE.match(lines[idx].strip()):
            player = lines[idx-1].strip() if idx-1 >= 0 else ""
            team_pos = lines[idx].strip()
            return player, team_pos
    return "", ""

def scrape_page_consensus(url, prop_label):
    driver.get(url)
    time.sleep(5)
    scroll_to_bottom_safely(driver, pause=2)

    body_text = driver.find_element(By.TAG_NAME, "body").text
    # Split into player blocks; Consensus pair appears at the END of each block before "View Matchup"
    blocks = re.split(r"\bView Matchup\b", body_text)

    rows = []
    for blk in blocks:
        # Quick filter: must have a TEAM - POS line, an 'O ' and a 'U '
        if " - " not in blk or "O " not in blk or "U " not in blk:
            continue

        player, team_pos = extract_player_teampos(blk)
        if not player or not team_pos:
            continue

        pairs = extract_all_pairs(blk)
        if not pairs:
            continue

        # Consensus = last valid pair in the block
        line_val, over_odds, under_odds = pairs[-1]

        rows.append({
            "Player": player,
            "Team_Pos": team_pos,
            "Prop Type": prop_label,
            "Prop Number": float(line_val),
            "Over Odds": int(over_odds),
            "Under Odds": int(under_odds),
        })

    return pd.DataFrame(rows)

# ====== RUN ======
all_frames = []
try:
    for prop_label, url in URLS:
        print(f"\n=== Scraping {prop_label} (Consensus via last-pair) ===")
        df = scrape_page_consensus(url, prop_label)
        print(f"✅ Extracted {len(df)} {prop_label} consensus props")
        path = out_dir / f"{prop_label.replace(' ','_').lower()}_consensus.csv"
        df.to_csv(path, index=False)
        print(f"💾 Saved {path.resolve()}")
        all_frames.append(df)

    combined = pd.concat(all_frames, ignore_index=True) if all_frames else pd.DataFrame()
    combined_path = out_dir / "all_markets_consensus.csv"
    combined.to_csv(combined_path, index=False)
    print(f"\n🎯 Combined consensus saved to {combined_path.resolve()}")

finally:
    driver.quit()



##Antime TD Scrape 

# Loop ALL Week 1 Anytime TD matchup pages on BettingPros and save CSVs

SEASON, WEEK = 2025, 1
OUT_DIR = Path(f"bettingpros_week{WEEK}_{SEASON}_anytime_td"); OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---- Week 1 slugs (away-vs-home) ----
MATCHUP_SLUGS = [
    "cowboys-vs-eagles",
    "chiefs-vs-chargers",
    "raiders-vs-patriots",
    "steelers-vs-jets",
    "buccaneers-vs-falcons",
    "dolphins-vs-colts",
    "cardinals-vs-saints",
    "giants-vs-commanders",
    "panthers-vs-jaguars",
    "bengals-vs-browns",
    "titans-vs-broncos",
    "49ers-vs-seahawks",
    "lions-vs-packers",
    "texans-vs-rams",
    "ravens-vs-bills",
    "vikings-vs-bears",
]

BASE = "https://www.bettingpros.com/nfl/odds/player-props/touchdown-scored"

# ---- Selenium setup (same as before) ----
DRIVER_PATH = "/Users/nicholashazzard/Downloads/chromedriver"  # your working path
opts = Options(); opts.add_argument("--headless=new"); opts.add_argument("--disable-gpu")
opts.add_argument("--window-size=1920,1080"); opts.add_argument("--no-sandbox"); opts.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(DRIVER_PATH), options=opts)

def scroll_bottom(drv, pause=1.0, loops=40):
    last = drv.execute_script("return document.body.scrollHeight")
    for _ in range(loops):
        drv.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new = drv.execute_script("return document.body.scrollHeight")
        if new == last: break
        last = new

TEAM_POS_RE   = re.compile(r"^[A-Z]{2,3}\s*-\s*[A-Z]{1,3}\b")
BOOK_LOGO_RE  = re.compile(r"^Logo for (.+)$", re.IGNORECASE)
ODDS_TOKEN_RE = re.compile(r"^(?:[+-]\d{2,4}|NL|OFF)$")

NOISE_PREFIXES = tuple(s.lower() for s in [
    "headshot of", "logo for", "nfl", "matchups", "markets", "weeks", "wager",
    "odds search", "search player", "open", "best odds", "consensus",
    "view matchup", "bettingpros logo", "mlb", "ncaaf", "careers", "privacy", "©"
])

def _prev_non_noise(lines, start_idx):
    j = start_idx
    while j >= 0:
        cand = lines[j].strip()
        if cand and not cand.lower().startswith(NOISE_PREFIXES):
            return cand
        j -= 1
    return ""

def parse_anytime_td(page_text: str):
    lines = [ln.strip() for ln in page_text.split("\n") if ln.strip()]
    n = len(lines)

    # 1) Books (ordered, de-duped)
    books = []
    for ln in lines:
        m = BOOK_LOGO_RE.match(ln)
        if m:
            books.append(m.group(1).strip())
    if any(ln == "Consensus" for ln in lines) and "Consensus" not in books:
        books.append("Consensus")
    seen = set(); books = [b for b in books if not (b in seen or seen.add(b))]

    # 2) Players (ordered) — robust backtracking for names
    players = []
    for i in range(n):
        if TEAM_POS_RE.match(lines[i]):
            team, pos = [s.strip() for s in lines[i].split("-", 1)]
            name = _prev_non_noise(lines, i-1)
            if name:
                players.append((name, team, pos))

    P = len(players)
    if P == 0:
        return pd.DataFrame(), books, players

    # 3) All odds tokens, in page order
    odds_tokens = [ln for ln in lines if ODDS_TOKEN_RE.match(ln)]

    # 4) Chunk into per-book lists of length P (pad last chunk if short)
    chunks = []
    for k in range(0, len(odds_tokens), P):
        chunk = odds_tokens[k:k+P]
        if len(chunk) < P:
            chunk += [""] * (P - len(chunk))
        chunks.append(chunk)

    # Align books ↔ chunks
    if len(chunks) < len(books):
        books = books[:len(chunks)]
    elif len(chunks) > len(books):
        books += [f"Book{idx}" for idx in range(len(books)+1, len(chunks)+1)]

    # 5) Build DataFrame
    rows = []
    for idx, (name, team, pos) in enumerate(players):
        row = {"Player": name, "Team": team, "Pos": pos}
        for bname, ch in zip(books, chunks):
            row[bname] = ch[idx] if idx < len(ch) else ""
        rows.append(row)

    df = pd.DataFrame(rows).replace({"NL": pd.NA, "OFF": pd.NA})
    for b in books:
        if b in df.columns:
            df[b] = pd.to_numeric(df[b], errors="ignore")
    return df, books, players

all_dfs = []
try:
    for slug in MATCHUP_SLUGS:
        url = f"{BASE}/{slug}/?season={SEASON}&week={WEEK}"
        print(f"\n=== {slug} ===")
        driver.get(url)
        time.sleep(4)
        scroll_bottom(driver, pause=1.0)

        page_text = driver.find_element(By.TAG_NAME, "body").text
        df, books, players = parse_anytime_td(page_text)

        if df.empty:
            print(f"  ! No rows parsed for {slug}. Skipping.")
            continue

        # Context + save
        df["market"] = "Anytime TD"; df["matchup"] = slug
        df["season"] = SEASON; df["week"] = WEEK

        path = OUT_DIR / f"{slug}_anytime_td.csv"
        df.to_csv(path, index=False)
        print(f"  -> saved {path.name} ({len(df)} players, {len(books)} books)")

        # peek a bit if you're in a notebook
        try:
            display(df.head(10))
        except:
            pass

        all_dfs.append(df)

    if all_dfs:
        combined = pd.concat(all_dfs, ignore_index=True)
        combo_path = OUT_DIR / "anytime_td_all_matchups.csv"
        combined.to_csv(combo_path, index=False)
        print(f"\n✅ Combined saved: {combo_path.resolve()} ({combined.shape[0]} rows)")
    else:
        print("\nNo matchup data parsed.")
finally:
    driver.quit()




## CSV Management - Combine and Clean Data

SEASON = 2025
WEEK = 1
rush_rec_df = pd.read_csv(f"bettingpros_week{WEEK}_{SEASON}/all_markets_consensus.csv")
anytime_df = pd.read_csv(f"bettingpros_week{WEEK}_{SEASON}_anytime_td/anytime_td_all_matchups.csv")


cols1 = ['Prop Number', 'Over Odds', "Under Odds"] 
rush_rec_df = rush_rec_df.replace("EVEN", "100")
for c in cols1:
    rush_rec_df[c] = pd.to_numeric(rush_rec_df[c], errors='coerce')


## Change Book if Necessary -- Using Consensus Book 7 for Anytime TDs
cols2 = ['Book13']
anytime_df = anytime_df.replace('NaN', 0)
for c in cols2:
    anytime_df[c] = pd.to_numeric(anytime_df[c], errors='coerce')


# Point Projections for Receptions, Rush Yards, Recieving Yards Function
def implied_points(row):
    if row['Prop Type'] == 'REC':
        if row['Over Odds'] < 0:
            row['Proj'] = 1*(row['Prop Number'] + ( abs((row['Over Odds']))/( (abs(row['Over Odds'])) + 100) - 0.5)  )
        elif row['Over Odds'] > 0:
            row['Proj'] = 1*(row['Prop Number'] + (100/(row['Over Odds'] + 100) - 0.5))
        
        else:
            row['Proj'] = 1* row['Prop Number']

    if row['Prop Type'] == 'RUSH YDS':
        if row['Over Odds'] < 0:
            row['Proj'] = 0.1 * (row['Prop Number'] + ( abs((row['Over Odds']))/( (abs(row['Over Odds'])) + 100) - 0.5)  )
        elif row['Over Odds'] > 0:
            row['Proj'] = 0.1*(row['Prop Number'] + (100/(row['Over Odds'] + 100) - 0.5))
        else:
            row['Proj'] = 0.1*row['Prop Number']

    if row['Prop Type'] == 'REC YDS':
        if row['Over Odds'] < 0:
            row['Proj'] = 0.1*(row['Prop Number'] + ( abs((row['Over Odds']))/( (abs(row['Over Odds'])) + 100) - 0.5)  )
        elif row['Over Odds'] > 0:
            row['Proj'] = 0.1*(row['Prop Number'] + (100/(row['Over Odds'] + 100) - 0.5))
        else:
            row['Proj'] = 0.1*row['Prop Number']
    return row
        


## Change Book if Necessary -- Using Consensus Book 7 for Anytime TDs
#Point Projection for Anytime TDs Function
def anytime_tds(row):
    if row['Book13'] < 0:
        row['Proj TD Pts'] = 6*( (abs(row['Book13'])/(abs(row['Book13']) + 100)))
    elif row['Book13'] > 0:
        row['Proj TD Pts'] = 6*((100/(row['Book13'] + 100)))
    
    return row



rush_rec_df = rush_rec_df.apply(implied_points, axis=1)
anytime_df = anytime_df.apply(anytime_tds, axis=1)





=== Scraping REC (Consensus via last-pair) ===
✅ Extracted 160 REC consensus props
💾 Saved /Users/nicholashazzard/FL/src/fantasyline/bettingpros_week1_2025/rec_consensus.csv

=== Scraping RUSH YDS (Consensus via last-pair) ===
✅ Extracted 99 RUSH YDS consensus props
💾 Saved /Users/nicholashazzard/FL/src/fantasyline/bettingpros_week1_2025/rush_yds_consensus.csv

=== Scraping REC YDS (Consensus via last-pair) ===
✅ Extracted 185 REC YDS consensus props
💾 Saved /Users/nicholashazzard/FL/src/fantasyline/bettingpros_week1_2025/rec_yds_consensus.csv

🎯 Combined consensus saved to /Users/nicholashazzard/FL/src/fantasyline/bettingpros_week1_2025/all_markets_consensus.csv

=== cowboys-vs-eagles ===
  -> saved cowboys-vs-eagles_anytime_td.csv (49 players, 13 books)


/var/folders/bp/bnbqqwq559j1jyrgpd5z1f9w0000gn/T/ipykernel_481/3031415264.py:335: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[b] = pd.to_numeric(df[b], errors="ignore")


,Player,Team,Pos,Book1,Book2,Book3,Book4,Book5,Book6,Book7,Book8,Book9,Book10,Book11,Book12,Book13,market,matchup,season,week
0,S. Barkley,PHI,RB,-205,-180.0,-194.0,-200.0,-180.0,NaN,NaN,-195.0,-200.0,NaN,NaN,NaN,-194,Anytime TD,cowboys-vs-eagles,2025,1
1,J. Hurts,PHI,QB,-115,-145.0,-148.0,-163.0,-150.0,NaN,NaN,-150.0,-145.0,NaN,NaN,NaN,-150,Anytime TD,cowboys-vs-eagles,2025,1
2,C. Lamb,DAL,WR,140,158.0,158.0,150.0,145.0,NaN,NaN,150.0,150.0,NaN,NaN,NaN,150,Anytime TD,cowboys-vs-eagles,2025,1
3,A.J. Brown,PHI,WR,125,164.0,164.0,150.0,155.0,NaN,NaN,155.0,160.0,NaN,NaN,NaN,155,Anytime TD,cowboys-vs-eagles,2025,1
4,D. Smith,PHI,WR,160,200.0,200.0,162.0,195.0,NaN,NaN,190.0,200.0,NaN,NaN,NaN,190,Anytime TD,cowboys-vs-eagles,2025,1
5,J. Williams,DAL,RB,225,220.0,200.0,175.0,220.0,NaN,NaN,205.0,200.0,NaN,NaN,NaN,200,Anytime TD,cowboys-vs-eagles,2025,1
6,G. Pickens,DAL,WR,265,240.0,240.0,225.0,230.0,NaN,NaN,225.0,240.0,NaN,NaN,NaN,225,Anytime TD,cowboys-vs-eagles,2025,1
7,D. Goedert,PHI,TE,200,300.0,300.0,225.0,270.0,NaN,NaN,275.0,270.0,NaN,NaN,NaN,270,Anytime TD,cowboys-vs-eagles,2025,1
8,J. Ferguson,DAL,TE,360,333.0,310.0,333.0,255.0,NaN,NaN,285.0,300.0,NaN,NaN,NaN,285,Anytime TD,cowboys-vs-eagles,2025,1
9,Philadelphia Eagles,PHI,DST,425,520.0,520.0,NaN,475.0,NaN,NaN,NaN,475.0,NaN,NaN,NaN,475,Anytime TD,cowboys-vs-eagles,2025,1



=== chiefs-vs-chargers ===
  -> saved chiefs-vs-chargers_anytime_td.csv (45 players, 13 books)


/var/folders/bp/bnbqqwq559j1jyrgpd5z1f9w0000gn/T/ipykernel_481/3031415264.py:335: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[b] = pd.to_numeric(df[b], errors="ignore")


...

,Player,Team,Pos,Book1,Book2,Book3,Book4,Book5,Book6,Book7,Book8,Book9,Book10,Book11,Book12,Book13,market,matchup,season,week
0,D. Henry,BAL,RB,-195,-150.0,-105.0,-110.0,185.0,NaN,NaN,170.0,170.0,NaN,NaN,NaN,175.0,Anytime TD,ravens-vs-bills,2025,1
1,J. Allen,BUF,QB,-115,105.0,170.0,140.0,195.0,NaN,NaN,180.0,190.0,NaN,NaN,NaN,190.0,Anytime TD,ravens-vs-bills,2025,1
2,J. Cook,BUF,RB,-115,185.0,175.0,175.0,180.0,NaN,NaN,190.0,170.0,NaN,NaN,NaN,190.0,Anytime TD,ravens-vs-bills,2025,1
3,M. Andrews,BAL,TE,200,195.0,200.0,175.0,205.0,NaN,NaN,190.0,170.0,NaN,NaN,NaN,205.0,Anytime TD,ravens-vs-bills,2025,1
4,Z. Flowers,BAL,WR,190,200.0,185.0,200.0,240.0,NaN,NaN,205.0,240.0,NaN,NaN,NaN,240.0,Anytime TD,ravens-vs-bills,2025,1
5,L. Jackson,BAL,QB,165,205.0,185.0,200.0,235.0,NaN,NaN,240.0,240.0,NaN,NaN,NaN,280.0,Anytime TD,ravens-vs-bills,2025,1
6,K. Coleman,BUF,WR,175,240.0,250.0,240.0,350.0,NaN,NaN,280.0,270.0,NaN,NaN,NaN,310.0,Anytime TD,ravens-vs-bills,2025,1
7,K. Shakir,BUF,WR,180,250.0,250.0,240.0,380.0,NaN,NaN,310.0,400.0,NaN,NaN,NaN,425.0,Anytime TD,ravens-vs-bills,2025,1
8,R. Bateman,BAL,WR,195,350.0,280.0,300.0,425.0,NaN,NaN,400.0,350.0,NaN,NaN,NaN,400.0,Anytime TD,ravens-vs-bills,2025,1
9,D. Kincaid,BUF,TE,210,380.0,440.0,450.0,500.0,NaN,NaN,400.0,500.0,NaN,NaN,NaN,450.0,Anytime TD,ravens-vs-bills,2025,1



=== vikings-vs-bears ===
  -> saved vikings-vs-bears_anytime_td.csv (35 players, 13 books)


/var/folders/bp/bnbqqwq559j1jyrgpd5z1f9w0000gn/T/ipykernel_481/3031415264.py:335: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[b] = pd.to_numeric(df[b], errors="ignore")


,Player,Team,Pos,Book1,Book2,Book3,Book4,Book5,Book6,Book7,Book8,Book9,Book10,Book11,Book12,Book13,market,matchup,season,week
0,J. Jefferson,MIN,WR,120,135.0,120.0,120.0,135.0,NaN,NaN,125.0,125.0,NaN,NaN,NaN,120,Anytime TD,vikings-vs-bears,2025,1
1,A. Jones Sr.,MIN,RB,145,160.0,150.0,150.0,160.0,NaN,NaN,150.0,150.0,NaN,NaN,NaN,150,Anytime TD,vikings-vs-bears,2025,1
2,J. Mason,MIN,RB,150,165.0,160.0,162.0,165.0,NaN,NaN,165.0,160.0,NaN,NaN,NaN,165,Anytime TD,vikings-vs-bears,2025,1
3,D. Swift,CHI,RB,120,170.0,130.0,140.0,170.0,NaN,NaN,140.0,125.0,NaN,NaN,NaN,140,Anytime TD,vikings-vs-bears,2025,1
4,DJ Moore,CHI,WR,185,185.0,160.0,175.0,185.0,NaN,NaN,170.0,155.0,NaN,NaN,NaN,170,Anytime TD,vikings-vs-bears,2025,1
5,R. Odunze,CHI,WR,205,205.0,180.0,200.0,205.0,NaN,NaN,190.0,170.0,NaN,NaN,NaN,190,Anytime TD,vikings-vs-bears,2025,1
6,T.J. Hockenson,MIN,TE,265,225.0,185.0,210.0,225.0,NaN,NaN,205.0,175.0,NaN,NaN,NaN,205,Anytime TD,vikings-vs-bears,2025,1
7,R. Johnson,CHI,RB,185,300.0,300.0,260.0,295.0,NaN,NaN,280.0,290.0,NaN,NaN,NaN,280,Anytime TD,vikings-vs-bears,2025,1
8,A. Thielen,MIN,WR,650,310.0,310.0,275.0,310.0,NaN,NaN,305.0,290.0,NaN,NaN,NaN,310,Anytime TD,vikings-vs-bears,2025,1
9,C. Loveland,CHI,TE,300,380.0,360.0,333.0,380.0,NaN,NaN,360.0,340.0,NaN,NaN,NaN,360,Anytime TD,vikings-vs-bears,2025,1



✅ Combined saved: /Users/nicholashazzard/FL/src/fantasyline/bettingpros_week1_2025_anytime_td/anytime_td_all_matchups.csv (637 rows)


Updated CSV Code

In [2]:
# 1) Base projections from rushing/receiving
df_final = (
    rush_rec_df
    .groupby(["Player", "Team_Pos"], as_index=False)["Proj"]
    .sum()
    .rename(columns={"Proj": "Base_Projection"})
)

# --- NEW: split Team_Pos BEFORE merging so Team exists on the left
df_final[["Team", "Pos"]] = df_final["Team_Pos"].astype(str).str.split(" - ", n=1, expand=True)
df_final["Team"] = df_final["Team"].astype(str).str.upper().str.strip()
df_final["Pos"]  = df_final["Pos"].astype(str).str.strip()

# 2) Aggregate anytime TD points (ensure one row per Player/Team)
any_td = (
    anytime_df
    .assign(Team=lambda d: d["Team"].astype(str).str.upper().str.strip())
    .groupby(["Player", "Team"], as_index=False)["Proj TD Pts"]
    .sum()
)

# 3) Merge and compute total  ✅ now merge on Player + Team
df_final = df_final.merge(any_td, on=["Player", "Team"], how="left")
df_final["Proj TD Pts"] = df_final["Proj TD Pts"].fillna(0).round(2)

df_final["Base_Projection"] = df_final["Base_Projection"].round(2)
df_final["Total_Projection"] = (df_final["Base_Projection"] + df_final["Proj TD Pts"]).round(2)

# (optional) round & reorder
df_final = df_final[["Player", "Team", "Pos", "Base_Projection", "Proj TD Pts", "Total_Projection"]]

#remove partial QB projections
df_final = df_final[df_final['Pos'] != 'QB']

df_final.to_csv(f"bettingpros_week{WEEK}_{SEASON}_final.csv", index=False)


In [3]:
df = pd.read_csv('../fantasyline/bettingpros_week1_2025_final.csv')